In [43]:
import requests

API_KEY = 'VOTRE_CLÉ_API_AMBEE'
latitude = 48.8566   # Exemple : Paris
longitude = 2.3522

url = f'https://api.ambeedata.com/latest/pollen/by-lat-lng'
headers = {
    'x-api-key': '1abda428d8ab2136e2b6e8a85131ff222c3551f66c2e756202c49a27914dec09',
    'Content-type': 'application/json'
}
params = {
    'lat': latitude,
    'lng': longitude
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Erreur: {response.status_code} - {response.text}")

{'message': 'success', 'lat': 48.8566, 'lng': 2.3522, 'data': [{'timezone': 'Europe/Paris', 'Species': {'Grass': {'Grass / Poaceae': 0}, 'Others': 2, 'Tree': {'Alder': 0, 'Birch': 0, 'Cypress': 0, 'Elm': 0, 'Hazel': 0, 'Oak': 0, 'Pine': 0, 'Plane': 0, 'Poplar / Cottonwood': 0}, 'Weed': {'Chenopod': 2, 'Mugwort': 1, 'Nettle': 28, 'Ragweed': 9}}, 'Risk': {'grass_pollen': 'Low', 'tree_pollen': 'Low', 'weed_pollen': 'Moderate'}, 'Count': {'grass_pollen': 0, 'tree_pollen': 0, 'weed_pollen': 42}, 'updatedAt': '2024-09-29T16:00:00.000Z'}]}


In [28]:
print(response)

<Response [200]>


In [47]:
import requests

API_KEY = 'YOUR_API_KEY'  # Replace 'YOUR_API_KEY' with your actual Ambee API key
latitude = 48.7643   # Latitude for Châtenay-Malabry
longitude = 2.2901   # Longitude for Châtenay-Malabry

url = 'https://api.ambeedata.com/latest/pollen/by-lat-lng'
headers = {
    'x-api-key': API_KEY,
    'Content-type': 'application/json'
}
params = {
    'lat': latitude,
    'lng': longitude
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    # Extract the tree pollen risk from the 'Risk' dictionary
    tree_pollen_risk = data['data'][0]['Risk']['tree_pollen'] if data['data'] else "No data available"
    print(f"Tree Pollen Risk: {tree_pollen_risk}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Tree Pollen Risk: Low
